In [1]:
import numpy as np
import pandas as pd
import difflib
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [2]:
movies=pd.read_csv('/content/movies.csv')

In [3]:
movies.head()

,index,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,...,runtime,spoken_languages,status,tagline,title,vote_average,vote_count,cast,crew,director
0,0,237000000,Action Adventure Fantasy Science Fiction,http://www.avatarmovie.com/,19995,culture clash future space war space colony so...,en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,...,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800,Sam Worthington Zoe Saldana Sigourney Weaver S...,"[{'name': 'Stephen E. Rivkin', 'gender': 0, 'd...",James Cameron
1,1,300000000,Adventure Fantasy Action,http://disney.go.com/disneypictures/pirates/,285,ocean drug abuse exotic island east india trad...,en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,...,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500,Johnny Depp Orlando Bloom Keira Knightley Stel...,"[{'name': 'Dariusz Wolski', 'gender': 2, 'depa...",Gore Verbinski
2,2,245000000,Action Adventure Crime,http://www.sonypictures.com/movies/spectre/,206647,spy based on novel secret agent sequel mi6,en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,...,148.0,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",Released,A Plan No One Escapes,Spectre,6.3,4466,Daniel Craig Christoph Waltz L\u00e9a Seydoux ...,"[{'name': 'Thomas Newman', 'gender': 2, 'depar...",Sam Mendes
3,3,250000000,Action Crime Drama Thriller,http://www.thedarkknightrises.com/,49026,dc comics crime fighter terrorist secret ident...,en,The Dark Knight Rises,Following the death of District Attorney Harve...,112.312950,...,165.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,The Legend Ends,The Dark Knight Rises,7.6,9106,Christian Bale Michael Caine Gary Oldman Anne ...,"[{'name': 'Hans Zimmer', 'gender': 2, 'departm...",Christopher Nolan
4,4,260000000,Action Adventure Science Fiction,http://movies.disney.com/john-carter,49529,based on novel mars medallion space travel pri...,en,John Carter,"John Carter is a war-weary, former military ca...",43.926995,...,132.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"Lost in our world, found in another.",John Carter,6.1,2124,Taylor Kitsch Lynn Collins Samantha Morton Wil...,"[{'name': 'Andrew Stanton', 'gender': 2, 'depa...",Andrew Stanton


In [4]:
movies.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4803 entries, 0 to 4802
Data columns (total 24 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   index                 4803 non-null   int64  
 1   budget                4803 non-null   int64  
 2   genres                4775 non-null   object 
 3   homepage              1712 non-null   object 
 4   id                    4803 non-null   int64  
 5   keywords              4391 non-null   object 
 6   original_language     4803 non-null   object 
 7   original_title        4803 non-null   object 
 8   overview              4800 non-null   object 
 9   popularity            4803 non-null   float64
 10  production_companies  4803 non-null   object 
 11  production_countries  4803 non-null   object 
 12  release_date          4802 non-null   object 
 13  revenue               4803 non-null   int64  
 14  runtime               4801 non-null   float64
 15  spoken_languages     

Most of the features in the data are string objects.
And many have null values.

# Pre-Processing the data

In [49]:
# selecting the relevant features for recommendation

selected_features = ['genres','keywords','tagline','cast','director','overview']

# replacing the null valuess with null string

for feature in selected_features:
  movies[feature] = movies[feature].fillna('')

In [50]:
# combining all the selected features

combined_features = movies['genres']+' '+movies['keywords']+' '+movies['tagline']+' '+movies['cast']+' '+movies['director']+' '+movies['overview']

In [51]:
print(type(combined_features))
combined_features.shape

<class 'pandas.core.series.Series'>


(4803,)

**The combined features is a pandas series.**

In [52]:
combined_features[0]

'Action Adventure Fantasy Science Fiction culture clash future space war space colony society Enter the World of Pandora. Sam Worthington Zoe Saldana Sigourney Weaver Stephen Lang Michelle Rodriguez James Cameron In the 22nd century, a paraplegic Marine is dispatched to the moon Pandora on a unique mission, but becomes torn between following orders and protecting an alien civilization.'

In [53]:
# converting the text data to feature vectors

vectorizer = TfidfVectorizer()

In [54]:
feature_vectors = vectorizer.fit_transform(combined_features)

In [55]:
print(feature_vectors)
feature_vectors.shape

  (0, 5245)	0.16506865163441017
  (0, 1014)	0.12285843797047788
  (0, 1243)	0.05270709034262742
  (0, 21523)	0.16650106431669984
  (0, 1281)	0.0319519027005067
  (0, 19650)	0.16120557402371172
  (0, 10368)	0.138482776543016
  (0, 2974)	0.09974129089792808
  (0, 27515)	0.1456922041909068
  (0, 2685)	0.09805341831299118
  (0, 4129)	0.06195738171764886
  (0, 18025)	0.1093940665047336
  (0, 28597)	0.15490753406550084
  (0, 19541)	0.04908322232312577
  (0, 18249)	0.14769810641761005
  (0, 27405)	0.031015019219557106
  (0, 7827)	0.17979882024342952
  (0, 14023)	0.04170092858483918
  (0, 17021)	0.14324534514274756
  (0, 20104)	0.19260031442011957
  (0, 4768)	0.12501550204808318
  (0, 239)	0.19716936546022965
  (0, 13474)	0.036478409857958735
  (0, 4288)	0.13213124585063998
  (0, 14180)	0.08418056281586365
  :	:
  (4802, 27153)	0.05940079357098276
  (4802, 19243)	0.061349419053434766
  (4802, 11941)	0.07829183421073846
  (4802, 29680)	0.07735547381237222
  (4802, 9346)	0.07104321122746965
  (4

(4803, 30592)

When we fit and transform a list of strings using **TfidfVectorizer()**, it returns a sparse matrix of shape **(num of samples, num of features)**, where "num of samples" is the number of documents/strings and "num of features" is the number of unique terms in the vocabulary created from the whole list.

In [56]:
# getting the similarity scores using cosine similarity

similarity = cosine_similarity(feature_vectors)

In [57]:
print(similarity)
print('\n')
print('\n')
print(similarity.shape)

[[1.         0.05083168 0.0332947  ... 0.02749812 0.0304889  0.0072518 ]
 [0.05083168 1.         0.04356836 ... 0.05077045 0.03100979 0.01521198]
 [0.0332947  0.04356836 1.         ... 0.02646984 0.04751623 0.01372603]
 ...
 [0.02749812 0.05077045 0.02646984 ... 1.         0.03481447 0.03546821]
 [0.0304889  0.03100979 0.04751623 ... 0.03481447 1.         0.03098945]
 [0.0072518  0.01521198 0.01372603 ... 0.03546821 0.03098945 1.        ]]




(4803, 4803)


**Cosine Similarity** takes one movie and calculates the similarity score for all other movie features present in the feature vectors. So **output is a square matrix.**

In [64]:
# getting the movie name from the user

movie_name = input(' Enter your favourite movie name : ')

 Enter your favourite movie name : Avengers


In [65]:
# creating a list with all the movie names given in the dataset

list_of_all_titles = movies['title'].tolist()
print(list_of_all_titles)


print("\nFINDING CLOSE MATCHES")
# finding the close match for the movie name given by the user

find_close_match = difflib.get_close_matches(movie_name, list_of_all_titles)
print(find_close_match)

# we do this because the input from the user may not be accurate with the names in the dataset. For instance the spelling entered by the user maybe incorrect

['Avatar', "Pirates of the Caribbean: At World's End", 'Spectre', 'The Dark Knight Rises', 'John Carter', 'Spider-Man 3', 'Tangled', 'Avengers: Age of Ultron', 'Harry Potter and the Half-Blood Prince', 'Batman v Superman: Dawn of Justice', 'Superman Returns', 'Quantum of Solace', "Pirates of the Caribbean: Dead Man's Chest", 'The Lone Ranger', 'Man of Steel', 'The Chronicles of Narnia: Prince Caspian', 'The Avengers', 'Pirates of the Caribbean: On Stranger Tides', 'Men in Black 3', 'The Hobbit: The Battle of the Five Armies', 'The Amazing Spider-Man', 'Robin Hood', 'The Hobbit: The Desolation of Smaug', 'The Golden Compass', 'King Kong', 'Titanic', 'Captain America: Civil War', 'Battleship', 'Jurassic World', 'Skyfall', 'Spider-Man 2', 'Iron Man 3', 'Alice in Wonderland', 'X-Men: The Last Stand', 'Monsters University', 'Transformers: Revenge of the Fallen', 'Transformers: Age of Extinction', 'Oz: The Great and Powerful', 'The Amazing Spider-Man 2', 'TRON: Legacy', 'Cars 2', 'Green Lant

In [66]:
close_match = find_close_match[0]
print(close_match)

# finding the index of the movie with title

index_of_movie = movies[movies.title == close_match]['index'].values[0]
print(index_of_movie)

The Avengers
16


In [67]:
# getting list of similarity scores with all other movies in the dataset

similarity_score = list(enumerate(similarity[index_of_movie]))
print(similarity_score)
print('\n')
# the length of the list is the number of movies present in the dataset
print(len(similarity_score))

[(0, 0.04270511084643852), (1, 0.06224849736436751), (2, 0.038497022076105974), (3, 0.03866029496208938), (4, 0.07911855483427592), (5, 0.028219968789126127), (6, 0.03223768203133271), (7, 0.44023537999611306), (8, 0.05694612067285467), (9, 0.0900244307215416), (10, 0.07257687378091392), (11, 0.024244462485105166), (12, 0.021581747144618836), (13, 0.030140555006309223), (14, 0.12280045566128427), (15, 0.04412560274546139), (16, 1.0000000000000002), (17, 0.0621768846858582), (18, 0.09406180183659207), (19, 0.046170896553890076), (20, 0.07491686150195936), (21, 0.05098295245244174), (22, 0.02999812856331277), (23, 0.04039353374924208), (24, 0.03960799005044994), (25, 0.029469554253520047), (26, 0.20395712302219832), (27, 0.07126731298189382), (28, 0.05668291840266033), (29, 0.06763120531181949), (30, 0.08646957595640709), (31, 0.13988291470369982), (32, 0.04708294714925445), (33, 0.13338190411264159), (34, 0.014097601041070489), (35, 0.03929614190478607), (36, 0.05639013876033423), (37, 

In [68]:
# sorting the movies based on their similarity score

sorted_similar_movies = sorted(similarity_score, key = lambda x:x[1], reverse = True)
print(sorted_similar_movies)

[(16, 1.0000000000000002), (7, 0.44023537999611306), (85, 0.22030391002885588), (79, 0.21875024386891997), (26, 0.20395712302219832), (511, 0.1822759407731296), (174, 0.17132075087923182), (126, 0.165340198355461), (64, 0.16420929069328924), (169, 0.1463207117759176), (46, 0.14225094695565857), (182, 0.1401795608196723), (31, 0.13988291470369982), (33, 0.13338190411264159), (68, 0.1320883910974722), (101, 0.12943581971790338), (129, 0.12929891610324507), (203, 0.12580808444072306), (14, 0.12280045566128427), (38, 0.12056450312255527), (1720, 0.11700903976544032), (870, 0.11672626627773836), (1294, 0.11388770846814351), (94, 0.11336879003815323), (1230, 0.11256437839498352), (215, 0.11113767691337564), (788, 0.10986510020201289), (1892, 0.1091073911294834), (782, 0.10737096085522715), (713, 0.10679787508044816), (4401, 0.10525669069412166), (1569, 0.10467168495538937), (242, 0.1031335794946852), (4759, 0.10017500295475637), (2874, 0.09897325494600732), (1286, 0.09838660260231001), (256,

In [75]:
# print the name of similar movies based on the index

print('Movies suggested for YOU\n')
print('How many similar movies do you want the system to recommend? \n')
n=int(input('Enter a number : \n'))

for i in range(0,n):
  idx=sorted_similar_movies[i][0]
  title_from_index=movies[movies['index']==idx]['title'].values[0]
  print(i+1, ': ', title_from_index)

Movies suggested for you : 

How many similar movies do you want the system to recommend? 

Enter a number : 
35
1 :  The Avengers
2 :  Avengers: Age of Ultron
3 :  Captain America: The Winter Soldier
4 :  Iron Man 2
5 :  Captain America: Civil War
6 :  X-Men
7 :  The Incredible Hulk
8 :  Thor: The Dark World
9 :  X-Men: Apocalypse
10 :  Captain America: The First Avenger
11 :  X-Men: Days of Future Past
12 :  Ant-Man
13 :  Iron Man 3
14 :  X-Men: The Last Stand
15 :  Iron Man
16 :  X-Men: First Class
17 :  Thor
18 :  X2
19 :  Man of Steel
20 :  The Amazing Spider-Man 2
21 :  Kick-Ass
22 :  Superman II
23 :  Serenity
24 :  Guardians of the Galaxy
25 :  The Shadow
26 :  Fantastic 4: Rise of the Silver Surfer
27 :  Deadpool
28 :  The Losers
29 :  The Spirit
30 :  The Horse Whisperer
31 :  The Helix... Loaded
32 :  Down to Earth
33 :  Fantastic Four
34 :  The Image Revolution
35 :  In the Cut


# MOVIE RECOMMENDATION SYSTEM

In [84]:
name= input(' Enter the name of the movie :\n')

list_movies=movies['title'].tolist()

close_matches=difflib.get_close_matches(name,list_movies)

closest_match=close_matches[0]

index_movie=movies[movies['title']==closest_match]['index'].values[0]

similarity_scoress=list(enumerate(similarity[index_movie]))

sorted_movies = sorted(similarity_scoress, key = lambda x:x[1], reverse = True)
print(sorted_movies)
print('Movies suggested for YOU\n')
print('How many similar movies do you want the system to recommend? \n')
n=int(input('Enter a number : \n'))

for i in range(0,n):
  idx=sorted_movies[i][0]
  title_from_index=movies[movies['index']==idx]['title'].values[0]
  print(i+1, ': ', title_from_index)

 Enter the name of the movie :
Pulp Fiction
[(3232, 1.0000000000000007), (264, 0.13678577506636164), (476, 0.126554515295702), (4620, 0.11320816895561563), (4648, 0.10602585930055264), (1964, 0.10361383829044962), (828, 0.10336863257493611), (1307, 0.1015246873710107), (3491, 0.09892762483565373), (684, 0.09884773343617612), (1005, 0.09545114458670646), (3636, 0.09514685180833717), (830, 0.09476801326596655), (1557, 0.09432579516059225), (4624, 0.09256786765677691), (1949, 0.09191035380470948), (973, 0.09149438320736641), (1038, 0.09045022757096421), (18, 0.09010049763281368), (1952, 0.08982075073622639), (2917, 0.08979495878564629), (527, 0.08961464143851634), (4240, 0.08897819079904824), (355, 0.08818363199178048), (1782, 0.08792246238522067), (4019, 0.08787315891912434), (4404, 0.0877670023319857), (873, 0.0876998465854096), (4033, 0.08737495301168244), (2553, 0.08701061312531397), (2050, 0.08655650666957207), (1233, 0.08602176103615569), (3859, 0.08449564425505851), (380, 0.0838850